In [0]:

import keras
keras.__version__
from keras.applications import MobileNet

conv_base = MobileNet(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224,3))


import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import time
#base_dir = '/Users/fchollet/Downloads/cats_and_dogs_small'

#train_dir = os.path.join(base_dir, 'train')
#validation_dir = os.path.join(base_dir, 'validation')
#test_dir = os.path.join(base_dir, 'test')

train_dir = '/content/drive/My Drive/TechField/HotDogData/train'
validation_dir = '/content/drive/My Drive/TechField/HotDogData/test'
#conv_base.summary()

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
start = time.time()
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 1024))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels
  
  
 
train_features, train_labels = extract_features(train_dir, 498)
validation_features, validation_labels = extract_features(validation_dir, 500)
#test_features, test_labels = extract_features(test_dir, 1000)


train_features = np.reshape(train_features, (498, 7 * 7 * 1024))
validation_features = np.reshape(validation_features, (500, 7 * 7 * 1024))
#test_features = np.reshape(test_features, (1000, 4 * 4 * 512))




from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=7 * 7 * 1024))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.Nadam(lr=2e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=60,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))
end = time.time()
print(end - start)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
17227776/17225924 [==============================] - 1s 0us/step


FileNotFoundError: ignored